An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be reversed, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits reversed:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits reversed:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits reversed:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs

In [1]:
import numpy as np
from six.moves import range
import sys

In [2]:
import keras as K
import tensorflow as tf
from keras.models import Sequential
from keras import layers

# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
config = tf.ConfigProto(device_count = {'GPU': 1 , 'CPU': 8} ) 
config.gpu_options.allow_growth = True

sess = tf.Session(config=config) 
K.backend.set_session(sess)

Using TensorFlow backend.


In [3]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.

        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.

        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [4]:
# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [5]:
ctable.encode(C='0 +', num_rows=10)

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [6]:
x = ctable.encode(C='0 +', num_rows=5)
ctable.decode(x)

'0 +  '

In [7]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [8]:
def generate_training(TRAINING_SIZE = 50000, DIGITS = 3, REVERSE = True):

    # Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
    # int is DIGITS.
    MAXLEN = DIGITS + 1 + DIGITS

    questions = []
    answers = []
    seen = set()
    print('Generating data...')
    while len(questions) < TRAINING_SIZE:
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b = f(), f()
        # Skip any addition questions we've already seen
        # Also skip any such that x+Y == Y+x (hence the sorting).
        key = tuple(sorted((a, b)))
        if key in seen:
            continue
        seen.add(key)
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a + b)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))
        if REVERSE:
            # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
            # space used for padding.)
            query = query[::-1]
        questions.append(query)
        answers.append(ans)
    print('Total addition questions:', len(questions))
    out = np.vstack((np.array(questions), np.array(answers)))
    return out.T

In [9]:
train3char = generate_training(TRAINING_SIZE=50000, DIGITS=3, REVERSE=False)

Generating data...
Total addition questions: 50000


In [10]:
train3char.shape

(50000, 2)

In [11]:
train3char

array([['6+8    ', '14  '],
       ['9+7    ', '16  '],
       ['644+360', '1004'],
       ...,
       ['38+949 ', '987 '],
       ['576+533', '1109'],
       ['13+818 ', '831 ']], dtype='<U7')

In [12]:
r = np.random.randint(low=0, high=train3char.shape[0], size=4)
train3char[r, :]

array([['8+170  ', '178 '],
       ['18+244 ', '262 '],
       ['73+899 ', '972 '],
       ['144+71 ', '215 ']], dtype='<U7')

In [13]:
def vectorization(train, DIGITS):
    MAXLEN = 2*DIGITS+1
    print('Vectorization...')
    x = np.zeros((train.shape[0], MAXLEN, len(chars)), dtype=np.bool)
    y = np.zeros((train.shape[0], DIGITS + 1, len(chars)), dtype=np.bool)
    # encode questions
    for i, sentence in enumerate(train[:, 0]):
        x[i] = ctable.encode(sentence, MAXLEN)
    # encode answers
    for i, sentence in enumerate(train[:, 1]):
        y[i] = ctable.encode(sentence, DIGITS + 1)

    # Shuffle (x, y) in unison as the later parts of x will almost all be larger
    # digits.
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    x = x[indices]
    y = y[indices]

    # Explicitly set apart 10% for validation data that we never train over.
    split_at = len(x) - len(x) // 10
    (x_train, x_val) = x[:split_at], x[split_at:]
    (y_train, y_val) = y[:split_at], y[split_at:]

    print('Shapes in training Data:')
    print(x_train.shape)
    print(y_train.shape)

    print('Shapes in validation Data:')
    print(x_val.shape)
    print(y_val.shape)
    return x_train, y_train, x_val, y_val

In [14]:
x_train_3char, y_train_3char, x_val_3char, y_val_3char = vectorization(train=train3char, DIGITS=3)

Vectorization...
Shapes in training Data:
(45000, 7, 12)
(45000, 4, 12)
Shapes in validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [15]:
def build_model(chars, rnn_type='gru', DIGITS=3, HIDDEN_SIZE=10, BATCH_SIZE=100, DECODER_LAYERS=1):
    MAXLEN = 2*DIGITS + 1
    if rnn_type.lower() == 'gru':
        RNN = layers.GRU
    elif rnn_type.lower() == 'lstm':
        RNN = layers.LSTM
    elif rnn_type.lower() == 'rnn':
        RNN = layers.SimpleRNN    
    else:
        print('{rnn_type} RNN type not covered'.format(rnn_type=rnn_type))
        sys.exit(0)
        
    print('Build model...')
    model = Sequential()
    # "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
    # Note: In a situation where your input sequences have a variable length,
    # use input_shape=(None, num_feature).
    model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
    # As the decoder RNN's input, repeatedly provide with the last output of
    # RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
    # length of output, e.g., when DIGITS=3, max output is 999+999=1998.
    model.add(layers.RepeatVector(DIGITS + 1))
    # The decoder RNN could be multiple layers stacked or a single layer.
    for _ in range(DECODER_LAYERS):
        # By setting return_sequences to True, return not only the last output but
        # all the outputs so far in the form of (num_samples, timesteps,
        # output_dim). This is necessary as TimeDistributed in the below expects
        # the first dimension to be the timesteps.
        model.add(RNN(HIDDEN_SIZE, return_sequences=True))

    # Apply a dense layer to the every temporal slice of an input. For each of step
    # of the output sequence, decide which character should be chosen.
    model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    return model

In [16]:
model3char = build_model(chars=chars, rnn_type='lstm', DIGITS=3, HIDDEN_SIZE=128, BATCH_SIZE=128, DECODER_LAYERS=1)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [17]:
def train(model, x_train, y_train, x_val, y_val,  n_epochs=20, BATCH_SIZE=128, REVERSE=False):
    # Train the model each generation and show predictions against the validation
    # dataset.
    for iteration in range(1, n_epochs+1):
        print()
        print('-' * 50)
        print('Iteration', iteration)
        model.fit(x_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=1,
                  validation_data=(x_val, y_val))
        # Select 10 samples from the validation set at random so we can visualize
        # errors.
        for i in range(5):
            ind = np.random.randint(0, len(x_val))
            rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
            preds = model.predict_classes(rowx, verbose=0)
            q = ctable.decode(rowx[0])
            correct = ctable.decode(rowy[0])
            guess = ctable.decode(preds[0], calc_argmax=False)
            print('Question: ', q[::-1] if REVERSE else q, end=' ')
            print('Answer: ', correct, end=' ')
            if correct == guess:
                print(colors.ok + '☑' + colors.close, end=' ')
            else:
                print(colors.fail + '☒' + colors.close, end=' ')
            print('Guess: ', guess, end='\n')

In [18]:
train(model3char, x_train_3char, y_train_3char, x_val_3char, y_val_3char,  n_epochs=100, 
      BATCH_SIZE=128, REVERSE=False)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 6s 135us/step - loss: 1.8939 - acc: 0.3216 - val_loss: 1.7996 - val_acc: 0.3349
Question:  5+940   Answer:  945  ☒ Guess:  109 
Question:  12+8    Answer:  20   ☒ Guess:  43  
Question:  312+1   Answer:  313  ☒ Guess:  43  
Question:  230+83  Answer:  313  ☒ Guess:  136 
Question:  22+408  Answer:  430  ☒ Guess:  126 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 5s 110us/step - loss: 1.7584 - acc: 0.3504 - val_loss: 1.6907 - val_acc: 0.3755
Question:  183+0   Answer:  183  ☒ Guess:  11  
Question:  9+217   Answer:  226  ☒ Guess:  11  
Question:  832+62  Answer:  894  ☒ Guess:  709 
Question:  195+55  Answer:  250  ☒ Guess:  501 
Question:  1+183   Answer:  184  ☒ Guess:  11  

-------------------

In [23]:
train5char = generate_training(TRAINING_SIZE=550000, DIGITS=5, REVERSE=False)

Generating data...
Total addition questions: 550000


In [24]:
r = np.random.randint(low=0, high=train5char.shape[0], size=4)
train5char[r, :]

array([['7197+208   ', '7405  '],
       ['155+72972  ', '73127 '],
       ['99+5467    ', '5566  '],
       ['9122+93    ', '9215  ']], dtype='<U11')

In [25]:
x_train_5char, y_train_5char, x_val_5char, y_val_5char = vectorization(train=train5char, DIGITS=5)

Vectorization...
Shapes in training Data:
(495000, 11, 12)
(495000, 6, 12)
Shapes in validation Data:
(55000, 11, 12)
(55000, 6, 12)


In [26]:
model5char = build_model(chars=chars, rnn_type='lstm', DIGITS=5, HIDDEN_SIZE=128, BATCH_SIZE=128, DECODER_LAYERS=1)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 6, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 6, 128)            131584    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 6, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [27]:
train(model5char, x_train_5char, y_train_5char, x_val_5char, y_val_5char,  n_epochs=30, 
      BATCH_SIZE=128, REVERSE=False)


--------------------------------------------------
Iteration 1
Train on 495000 samples, validate on 55000 samples
Epoch 1/1
495000/495000 [==============================] - 78s 158us/step - loss: 1.3338 - acc: 0.4996 - val_loss: 1.0341 - val_acc: 0.6049
Question:  692+462     Answer:  1154   ☒ Guess:  1105  
Question:  282+39      Answer:  321    ☒ Guess:  317   
Question:  849+44991   Answer:  45840  ☒ Guess:  45455 
Question:  1792+0      Answer:  1792   ☒ Guess:  1793  
Question:  54566+9     Answer:  54575  ☒ Guess:  54675 

--------------------------------------------------
Iteration 2
Train on 495000 samples, validate on 55000 samples
Epoch 1/1
495000/495000 [==============================] - 78s 158us/step - loss: 0.9227 - acc: 0.6450 - val_loss: 0.8438 - val_acc: 0.6733
Question:  7586+11213  Answer:  18799  ☒ Guess:  18876 
Question:  942+3       Answer:  945    ☒ Guess:  944   
Question:  84109+45722 Answer:  129831 ☒ Guess:  137667
Question:  20+495      Answer:  515    ☒ G

KeyboardInterrupt: 

Check Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs

In [ ]:
# geta data
train5char_reversed = generate_training(TRAINING_SIZE=550000, DIGITS=5, REVERSE=True)
x_train_5char_reversed, y_train_5char_reversed, x_val_5char_reversed, y_val_5char_reversed = \
    vectorization(train=train5char_reversed, DIGITS=5)

# build model
model5char_reversed = build_model(chars=chars, rnn_type='lstm', DIGITS=5, HIDDEN_SIZE=128, 
                                  BATCH_SIZE=128, DECODER_LAYERS=1)
# train model
train(model5char_reversed, x_train_5char_reversed, y_train_5char_reversed, x_val_5char_reversed, 
      y_val_5char_reversed,  n_epochs=30, BATCH_SIZE=128, REVERSE=True)